Install requirements

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\AMD\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [8]:
from bs4 import BeautifulSoup
from configparser import ConfigParser
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import requests
import json
import time
import pandas as pd
import torch

C:\Users\AMD\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configure authentication

In [10]:
config = ConfigParser()
config.read('config.ini')
MY_API_KEY = config['yelp']['APIKey']
HEADERS = {'Authorization': 'bearer %s' % MY_API_KEY}

Load model

In [79]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [5]:

# search business based on location
BUSINESS_PATH = 'https://api.yelp.com/v3/transactions/delivery/search'

# Define the Parameters of the search
PARAMETERS = {'location':'NYC'}

# Make a Request to the API, and return results
response = requests.get(url=BUSINESS_PATH, 
                        params=PARAMETERS, 
                        headers=HEADERS)

# Convert response to a JSON String
business_data = response.json()  

# print the data
print(json.dumps(business_data, indent = 3))

{
   "businesses": [
      {
         "id": "IuMvrvXTTZm4NQYO1O1dbg",
         "alias": "kuma-nyc-brooklyn-2",
         "name": "Kuma NYC",
         "image_url": "https://s3-media1.fl.yelpcdn.com/bphoto/s5SvRV7ZeCjeV4o_f2255Q/o.jpg",
         "is_closed": false,
         "url": "https://www.yelp.com/biz/kuma-nyc-brooklyn-2?adjust_creative=kaz4WEGsU5BDvYPUGdKMGQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_transactions_search_delivery&utm_source=kaz4WEGsU5BDvYPUGdKMGQ",
         "review_count": 60,
         "categories": [
            {
               "alias": "poke",
               "title": "Poke"
            },
            {
               "alias": "asianfusion",
               "title": "Asian Fusion"
            },
            {
               "alias": "ramen",
               "title": "Ramen"
            }
         ],
         "rating": 4.0,
         "coordinates": {
            "latitude": 40.699183,
            "longitude": -73.91304
         },
         "transactions": [
           

In [12]:
BUSINESS = 'https://api.yelp.com/v3/businesses/ippudo-ny-new-york-7'

# Make a Request to the API, and return results
response = requests.get(url=BUSINESS, headers=HEADERS)

# Convert response to a JSON String and fetch buisness Id
businessID = response.json()['id'] 

In [23]:
link = f"https://api.yelp.com/v3/businesses/{businessID}/reviews"
params = {'limit' : 50}
response = requests.get(url=link, headers=HEADERS, params=params)
response.json()

{'reviews': [{'id': 'umF4u6_FKAu2Brk2yo4QIg',
   'url': 'https://www.yelp.com/biz/ippudo-ny-new-york-7?adjust_creative=kaz4WEGsU5BDvYPUGdKMGQ&hrid=umF4u6_FKAu2Brk2yo4QIg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=kaz4WEGsU5BDvYPUGdKMGQ',
   'text': "This is the best ramen in New York. I've been here a numerous amounts of time and continuously come back for a delicious satisfying meal. On my last visit I...",
   'rating': 5,
   'time_created': '2023-01-08 18:18:14',
   'user': {'id': '_GrrYLSzyeeAO3o-SZrhwA',
    'profile_url': 'https://www.yelp.com/user_details?userid=_GrrYLSzyeeAO3o-SZrhwA',
    'image_url': None,
    'name': 'Crystal A.'}},
  {'id': '1I3irlfyKE5Ey7tOil3NWg',
   'url': 'https://www.yelp.com/biz/ippudo-ny-new-york-7?adjust_creative=kaz4WEGsU5BDvYPUGdKMGQ&hrid=1I3irlfyKE5Ey7tOil3NWg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=kaz4WEGsU5BDvYPUGdKMGQ',
   'text': 'My fav ramen spot in town!\n\nMy partner and I went la

In [24]:
[i['text'] for i in response.json()['reviews']]

["This is the best ramen in New York. I've been here a numerous amounts of time and continuously come back for a delicious satisfying meal. On my last visit I...",
 'My fav ramen spot in town!\n\nMy partner and I went last night for a casual dinner. We waited about 15 minutes to be seated. (Ippudo does not access...',
 'I had such a great time at Ippudo! Unfortunately they do not take reservations or call aheads, so when went on a Wednesday night I was told it would be a...']

API dont give the entire text. so its useless to continue with API calls

In [51]:
link = "https://www.yelp.com/biz/ippudo-ny-new-york-7"
webpage = requests.get(link).text
soup = BeautifulSoup(webpage, 'html.parser')
reviews = [i.text for i in soup.find_all('p', 'comment__09f24__gu0rG css-qgunke')]

using request to get the reviews

In [53]:
def getReviews(link):
    webpage = requests.get(link).text
    soup = BeautifulSoup(webpage, 'html.parser')
    return [i.text for i in soup.find_all('p', 'comment__09f24__gu0rG css-qgunke')]
    

Intial Request

In [56]:
reviews = []
intial_link = "https://www.yelp.com/biz/ippudo-ny-new-york-7"
reviews.extend(getReviews(intial_link))

Looping through remaining pages

In [63]:
for i in range(10, 100, 10):
    later_link = f"https://www.yelp.com/biz/ippudo-ny-new-york-7?start={i}"
    reviews.extend(getReviews(later_link)) 
    time.sleep(10) # waiting time of 10 secs before hitting the another request to avoid site crash and http ban

No. of reviews

In [65]:
len(reviews)

100

In [128]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    return int(torch.argmax(model(tokens).logits[0])) + 1

In [133]:
sentiment = []
labels = {
    1 : 'Bad',
    2 : 'Poor',
    3 : 'Average',
    4 : 'Great',
    5 : 'Excellent'
}
for i in reviews:
    sentiment.append(labels[sentiment_score(i[:512])]) # maximum tokens accpeted by the bert model

In [134]:
sentiments_df = pd.DataFrame()
sentiments_df['review'] = reviews
sentiments_df['sentiment'] = sentiment

In [135]:
sentiments_df

,review,sentiment
0,This is the best ramen in New York. I've been ...,Excellent
1,My fav ramen spot in town!My partner and I wen...,Excellent
2,I had such a great time at Ippudo! Unfortunate...,Great
3,The chicken appetizer I ordered was over fried...,Bad
4,This place was soo yummy and I recommend it to...,Great
...,...,...
95,Cold weather? Craving soup? This is the place ...,Average
96,Our friends had recommended this restaurant fo...,Great
97,Ippudo Ramen is what ramen restaurants should ...,Excellent
98,One of the best Japanese ramen in town! Used t...,Excellent


In [140]:
sentiments_df.iloc[99]['review']

"Good ramen and pork buns to go with a beer. It loses 1 star for NOT taking reservations and pretending to be a real Japanese restaurant with servers yelling their appreciation in Japanese. Real Japanese restaurants don't accept tip, vs the suggested minimum of 20% for bringing a few bowl of ramens, a pork bun & a beer to the table."